In [18]:
import json,pandas as pd
from driver import read, movies_path,reviews_path,get_imdbid_from_url

In [19]:
def movie_to_row(movie:dict):
    imdbid = get_imdbid_from_url(movie["url"])
    return dict(
        imdbid=imdbid,
        name=movie["name"],
        image=movie["image"],
        desc=movie["description"],
        ratingCount=movie["aggregateRating"]["ratingCount"],
        avgRating=movie["aggregateRating"]["ratingValue"],
        genre=movie["genre"],
        date=movie.get("datePublished",None),
        keywords=movie.get("keywords", None),
    )
rows = []
movies = read(movies_path)
for k in movies:
    if movies[k] is None or "aggregateRating" not in movies[k]:
        continue
    rows.append(movie_to_row(movies[k]))
movies = pd.DataFrame(rows)
# movies.genre
movies.rename(columns={"date":"movie_date"},inplace=True)
movies.genre = movies.genre.map(json.dumps)
movies.head()

,imdbid,name,image,desc,ratingCount,avgRating,genre,movie_date,keywords
0,tt0903747,Breaking Bad,https://m.media-amazon.com/images/M/MV5BMzU5ZG...,A chemistry teacher diagnosed with inoperable ...,2332753,9.5,"[""Crime"", ""Drama"", ""Thriller""]",2008-01-20,"cancer,chemistry,drug trade,drug,albuquerque n..."
1,tt5491994,Planet Earth II,https://m.media-amazon.com/images/M/MV5BMzY4ND...,David Attenborough returns with a new wildlife...,166556,9.4,"[""Documentary""]",2017-02-18,"nature documentary,mountain,desert,island,nature"
2,tt0795176,Planet Earth,https://m.media-amazon.com/images/M/MV5BY2NjND...,A documentary series on the wildlife found on ...,227445,9.4,"[""Documentary"", ""Family""]",2007-03-25,"animal,life,earth,tv mini series,nature"
3,tt0185906,Band of Brothers,https://m.media-amazon.com/images/M/MV5BMTQ3NT...,The story of Easy Company of the U.S. Army 101...,562189,9.4,"[""Action"", ""Drama"", ""History""]",2001-09-09,"world war two soldier,military,world war two,2..."
4,tt0111161,The Shawshank Redemption,https://m.media-amazon.com/images/M/MV5BMDAyY2...,A banker convicted of uxoricide forms a friend...,3045427,9.3,"[""Drama""]",1994-10-14,"prison,based on the works of stephen king,esca..."


In [20]:
def review_to_row(review, imdbid):
    return dict(
        id=review["id"],
        imdbid=imdbid,
        author=review["author"],
        title=review["title"],
        body=review["body"],
        dislikes=review["dislikes"],
        likes=review["likes"],
        date=review["date"],
        rating=review["rating"],
    )
rows = []
reviews = read(reviews_path)
for imdbid in reviews:
    for reviewid in reviews[imdbid]:
        rows.append(review_to_row(reviews[imdbid][reviewid], imdbid))
reviews = pd.DataFrame(rows)

In [21]:
merged_df = pd.merge(reviews, movies, on='imdbid', how='inner')
merged_df.to_csv("output/merged.csv",index=False)